# Calculating the potential revenue of each market

In [113]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [114]:
df = pd.read_csv('../data/cleaned_data/eu_da_prices_clean.csv', index_col=0)
df.date = pd.to_datetime(df.date)
df = df.loc[df.date >= pd.to_datetime('2022-01-01')]
df.head()

,date,hour,datetime,Germany,De-Lu,Belgium,Denmark 1,Denmark 2,France,Netherlands,...,Austria,Poland,Sweden 4,Switzerland,Czech,Italy,Slovenia,Hungary,GB,Greece
26304,2022-01-01,1,2022-01-01 00:00:00,50.05,78.44,82.02,50.05,46.60,89.06,124.70,...,61.84,59.03,46.60,119.97,50.05,170.28,61.84,61.84,81.87,170.28
26305,2022-01-01,2,2022-01-01 01:00:00,41.33,69.75,67.07,41.33,41.33,78.48,124.70,...,51.44,49.37,41.33,101.54,41.33,155.72,51.44,41.33,64.49,155.72
26306,2022-01-01,3,2022-01-01 02:00:00,43.22,68.04,75.11,43.22,42.18,85.16,134.00,...,55.91,43.22,42.18,52.13,43.22,147.09,55.91,43.22,24.99,147.09
26307,2022-01-01,4,2022-01-01 03:00:00,45.46,51.27,50.91,45.46,44.37,50.00,58.80,...,46.94,45.46,44.37,20.78,45.46,91.00,46.94,45.46,7.02,91.00
26308,2022-01-01,5,2022-01-01 04:00:00,37.67,42.46,37.67,37.67,37.67,37.67,37.67,...,37.67,37.67,37.67,15.66,37.67,104.00,37.67,37.67,2.74,104.00


### Single cycle 1 hour

In [115]:
cycle_1_hour_1 = df.drop(['hour', 'datetime'], axis=1).groupby('date').apply(lambda x: x.max() - x.min())
cycle_1_hour_1.head()

,date,Germany,De-Lu,Belgium,Denmark 1,Denmark 2,France,Netherlands,Norway 2,Austria,Poland,Sweden 4,Switzerland,Czech,Italy,Slovenia,Hungary,GB,Greece
date,,,,,,,,,,,,,,,,,,,
2022-01-01,0 days,112.30,107.41,112.30,112.30,112.30,112.30,112.30,39.27,112.30,112.30,112.30,133.23,112.30,146.67,112.30,112.30,225.96,174.04
2022-01-02,0 days,40.98,56.86,40.53,65.84,65.84,40.53,80.34,31.91,65.84,44.90,65.84,110.41,65.84,145.00,65.84,215.44,143.45,235.30
2022-01-03,0 days,149.54,137.56,163.95,139.47,130.33,180.30,215.05,31.49,141.42,100.65,119.75,223.51,141.42,216.73,285.29,173.00,186.21,188.91
2022-01-04,0 days,92.44,93.44,91.90,98.53,148.78,85.00,88.90,54.50,90.53,96.03,148.78,115.05,92.44,96.21,90.53,143.64,132.33,121.64
2022-01-05,0 days,72.71,113.59,165.17,98.56,98.56,189.62,239.87,44.72,104.18,53.37,98.56,151.03,94.62,189.62,104.18,104.18,116.59,160.80


In [116]:
cycle_1_hour_1_rev = cycle_1_hour_1.sum()
cycle_1_hour_1_rev = cycle_1_hour_1_rev.drop('date').astype(float).sort_values(ascending=False)
cycle_1_hour_1_rev

Greece         76838.35
Netherlands    74865.43
Hungary        74368.99
Sweden 4       73867.40
Belgium        72030.46
Denmark 2      70884.84
Germany        68252.06
Slovenia       67368.71
Czech          66943.99
GB             65669.67
Denmark 1      65157.15
France         63843.47
Austria        59681.02
Italy          58954.95
De-Lu          54973.29
Poland         52234.22
Switzerland    46144.21
Norway 2       38806.75
dtype: float64

### Dual cycle 1 hour

In [117]:
cycle_2_hour_1 = df.copy()
cycle_2_hour_1['am_pm'] = cycle_2_hour_1.hour.apply(lambda x: 'am' if x < 13 else 'pm')
cycle_2_hour_1_am = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'am']
cycle_2_hour_1_pm = cycle_2_hour_1.loc[cycle_2_hour_1.am_pm == 'pm']

In [118]:
cycle_2_hour_1 = cycle_2_hour_1.drop(['hour', 'datetime'], axis=1).groupby(['date', 'am_pm']).apply(lambda x: x.max() - x.min())
cycle_2_hour_1

Germany   De-Lu  Belgium  Denmark 1  Denmark 2  France  \
date       am_pm                                                           
2022-01-01 am       46.43   42.53    46.43      46.43      46.43   51.39   
           pm       64.81   56.64    64.81      66.86      66.86   64.81   
2022-01-02 am       40.53   40.13    40.53      40.53      40.53   40.53   
           pm       35.43   46.76    36.57      62.47      62.47   34.92   
2022-01-03 am      123.98  114.75   103.16     122.83     113.69  180.30   
...                   ...     ...      ...        ...        ...     ...   
2022-12-29 pm       49.42   37.01    51.91      49.42      13.95   46.37   
2022-12-30 am       28.51   34.99    29.09      28.51      28.51   38.40   
           pm       30.78   40.16    30.81      30.78      30.77   30.16   
2022-12-31 am        2.29   12.73     1.54       6.83       4.10    1.06   
           pm        3.78   28.42    27.80      63.26      21.81   20.37   

                  Netherlands  Norway 2  Austria  Poland  Sweden 4  \
date       am_pm                                                     
2022-01-01 am           96.33     21.45    46.43   46.43     46.43   
           pm           64.81     33.54    64.81   64.81     66.86   
2022-01-02 am           40.53     17.88    40.53   40.62     40.53   
           pm           76.97     31.16    62.47   38.36     62.47   
2022-01-03 am          130.95     11.88   124.40   99.62    103.11   
...                       ...       ...      ...     ...       ...   
2022-12-29 pm           85.79     32.87    48.06   17.02     12.23   
2022-12-30 am           28.65     62.83    54.63   35.58     28.51   
           pm           30.88    120.55    50.02   54.29     30.77   
2022-12-31 am            6.82     60.89    10.28   74.39      4.10   
           pm           93.39     11.41    15.36   74.18     21.81   

                  Switzerland   Czech   Italy  Slovenia  Hungary      GB  \
date       am_pm                                                           
2022-01-01 am          104.31   46.43  102.29     46.43    46.43   90.31   
           pm           78.17   64.81  114.66     64.81    64.81  143.62   
2022-01-02 am           72.67   40.53   75.08     40.53    40.53   99.42   
           pm           77.49   62.47  100.21     62.47   212.07  110.07   
2022-01-03 am          214.78  128.46  156.24    232.44   134.01   70.73   
...                       ...     ...     ...       ...      ...     ...   
2022-12-29 pm           37.51   45.18  116.00     70.15    80.04  140.46   
2022-12-30 am           32.40   45.97  106.14     53.39    64.00  140.50   
           pm           31.00   46.49   99.64     72.56   154.47  158.00   
2022-12-31 am           10.00    4.01   78.00      9.29    21.69  149.66   
           pm           15.68   29.80  145.81     46.86   234.51  173.06   

                  Greece  
date       am_pm          
2022-01-01 am     129.66  
           pm     150.70  
2022-01-02 am     146.02  
           pm     180.46  
2022-01-03 am     188.91  
...                  ...  
2022-12-29 pm     175.58  
2022-12-30 am     108.92  
           pm     187.54  
2022-12-31 am     176.95  
           pm     193.96  

[730 rows x 18 columns]

In [119]:
cycle_2_hour_1_rev = cycle_2_hour_1.sum().sort_values(ascending=False)
cycle_2_hour_1_rev

Netherlands    118202.57
Greece         115454.81
Sweden 4       112972.29
Belgium        112553.58
Hungary        111015.74
Denmark 2      105492.89
Germany        104827.20
Czech          104274.78
Slovenia       100646.10
Denmark 1       97049.84
GB              96427.40
France          93747.97
Austria         91186.56
Italy           86987.76
De-Lu           84288.07
Poland          76592.56
Switzerland     69728.75
Norway 2        58146.63
dtype: float64

### Single cycle 2 hour

(Dis)charging times based on average price profile

In [120]:
charge_3_4 = ['Poland', 'Sweden 4']
charge_4_5 = ['De-Lu', 'Denmark 2', 'France', 'Norway 2', 'Austria', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB', 'Greece']
charge_14_15 = ['Belgium', 'Denmark 1', 'Netherlands']

discharge_18_19 = ['GB']
discharge_19_20 = ['Denmark 2', 'Norway 2', 'Sweden 4']
discharge_20_21 = ['De-Lu', 'Belgium', 'Denmark 1', 'France', 'Netherlands', 'Austria', 'Poland', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'Greece']

df['charge_discharge'] = 0

def calculate_single_cycle_2_hour_rev(df):
    charge_3_4_cost = df[charge_3_4].loc[(df.hour == 3) | (df.hour == 4)].sum()
    charge_4_5_cost = df[charge_4_5].loc[(df.hour == 4) | (df.hour == 5)].sum()
    charge_14_15_cost = df[charge_14_15].loc[(df.hour == 14) | (df.hour == 15)].sum()
    discharge_18_19_rev = df[discharge_18_19].loc[(df.hour == 18) | (df.hour == 19)].sum()
    discharge_19_20_rev = df[discharge_19_20].loc[(df.hour == 19) | (df.hour == 20)].sum()
    discharge_20_21_rev = df[discharge_20_21].loc[(df.hour == 20) | (df.hour == 21)].sum()

    charge_cost = pd.concat([charge_3_4_cost, charge_4_5_cost, charge_14_15_cost], axis=0)
    discharge_rev = pd.concat([discharge_18_19_rev, discharge_19_20_rev, discharge_20_21_rev], axis=0)

    total_rev = discharge_rev - charge_cost

    return total_rev

cycle_1_hour_2_rev = calculate_single_cycle_2_hour_rev(df).sort_values(ascending=False)
cycle_1_hour_2_rev

Hungary        109157.07
GB             102684.70
Slovenia       100491.01
Greece          98996.23
Sweden 4        98037.09
Netherlands     93562.84
France          90023.46
Czech           89933.57
Belgium         86047.09
Austria         85339.86
Italy           85073.47
Denmark 2       81789.71
De-Lu           78160.33
Denmark 1       73885.56
Poland          71653.49
Switzerland     71004.23
Norway 2        41564.46
dtype: float64

### Dual cycle 2 hour

(Dis)charging times based on average price profile

In [121]:
charge_3_4 = ['Poland', 'Sweden 4']
charge_4_5 = ['De-Lu', 'Denmark 2', 'France', 'Norway 2', 'Austria', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB', 'Belgium', 'Denmark 1', 'Netherlands', 'Greece']
charge_14_15 = ['Belgium', 'Denmark 1', 'Netherlands', 'De-Lu', 'Denmark 2', 'Norway 2', 'Austria', 'Poland', 'Sweden 4', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'GB', 'Greece']
charge_15_16 = ['France']

discharge_8_9 = ['De-Lu', 'Belgium', 'Denmark 1', 'Denmark 2', 'Netherlands', 'Austria', 'Poland', 'Sweden 4', 'Switzerland', 'Czech', 'Slovenia', 'Hungary', 'GB', 'Greece']
discharge_9_10 = ['France', 'Norway 2', 'Italy']
discharge_18_19 = ['GB']
discharge_19_20 = ['Denmark 2', 'Norway 2', 'Sweden 4']
discharge_20_21 = ['De-Lu', 'Belgium', 'Denmark 1', 'France', 'Netherlands', 'Austria', 'Poland', 'Switzerland', 'Czech', 'Italy', 'Slovenia', 'Hungary', 'Greece']

df['charge_discharge'] = 0

def calculate_single_cycle_2_hour_rev(df):
    charge_3_4_cost = df[charge_3_4].loc[(df.hour == 3) | (df.hour == 4)].sum()
    charge_4_5_cost = df[charge_4_5].loc[(df.hour == 4) | (df.hour == 5)].sum()
    charge_14_15_cost = df[charge_14_15].loc[(df.hour == 14) | (df.hour == 15)].sum()
    charge_15_16_cost = df[charge_15_16].loc[(df.hour == 15) | (df.hour == 16)].sum()
    discharge_8_9_rev = df[discharge_8_9].loc[(df.hour == 8) | (df.hour == 9)].sum()
    discharge_9_10_rev = df[discharge_9_10].loc[(df.hour == 9) | (df.hour == 10)].sum()
    discharge_18_19_rev = df[discharge_18_19].loc[(df.hour == 18) | (df.hour == 19)].sum()
    discharge_19_20_rev = df[discharge_19_20].loc[(df.hour == 19) | (df.hour == 20)].sum()
    discharge_20_21_rev = df[discharge_20_21].loc[(df.hour == 20) | (df.hour == 21)].sum()

    charge_cost = pd.concat([charge_3_4_cost, charge_4_5_cost, charge_14_15_cost, charge_15_16_cost], axis=0).reset_index().groupby('index').sum()
    discharge_rev = pd.concat([discharge_8_9_rev, discharge_9_10_rev, discharge_18_19_rev, discharge_19_20_rev, discharge_20_21_rev], axis=0).reset_index().groupby('index').sum()

    total_rev = discharge_rev - charge_cost

    return total_rev

cycle_2_hour_2_rev = calculate_single_cycle_2_hour_rev(df).sort_values(by=0, ascending=False)
cycle_2_hour_2_rev

,0
index,
Netherlands,154473.69
Hungary,152026.51
Belgium,148824.17
Greece,144067.19
Czech,141544.86
Sweden 4,141221.52
Denmark 2,140135.54
GB,136589.51
Slovenia,136507.05


### Plots

In [124]:
fig = make_subplots(
    rows=2, cols=1,
    shared_yaxes=True,
    subplot_titles=('Single cycle', 'Dual cycle')
)

fig.add_trace(go.Bar(
    x=cycle_1_hour_1_rev.index,
    y=cycle_1_hour_1_rev.values/1000,
    marker=dict(color=cycle_1_hour_1_rev.values, colorscale='solar')
), row=1, col=1)

fig.add_trace(go.Bar(
    x=cycle_2_hour_1_rev.index,
    y=cycle_2_hour_1_rev.values/1000,
    marker=dict(color=cycle_2_hour_1_rev.values, colorscale='solar'),
), row=2, col=1)

fig.update_yaxes(title='k€/MW')
fig.update_traces(texttemplate='%{value: .3r}', textangle=0)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 potential revenue for a 1 hour battery',
    yaxis_title='k€/MW',
    showlegend=False,
    template='plotly_white',
    width=1200,
    height=800
)

fig.write_image('../figures/1-hour-rev.jpeg', scale=5, engine='orca')
fig

In [123]:
fig = make_subplots(
    rows=2, cols=1,
    shared_yaxes=True,
    subplot_titles=('Single cycle', 'Dual cycle')
)

fig.add_trace(go.Bar(
    x=cycle_1_hour_2_rev.index,
    y=cycle_1_hour_2_rev.values/1000,
    marker=dict(color=cycle_1_hour_2_rev.values, colorscale='solar'),
    name='Single cycle (left)'
), row=1, col=1)

fig.add_trace(go.Bar(
    x=cycle_2_hour_2_rev.index,
    y=cycle_2_hour_2_rev.values.reshape(-1)/1000,
    marker=dict(color=cycle_2_hour_2_rev.values.reshape(-1), colorscale='solar'),
    name='Dual cycle (right)'
), row=2, col=1)

fig.update_yaxes(title='k€/MW')
fig.update_traces(texttemplate='%{value: .3r}', textangle=0)
fig.update_coloraxes(showscale=False)
fig.update_layout(
    title='2022 revenue for a 2 hour battery with preset flow hours',
    showlegend=False,
    template='plotly_white',
    width=1200,
    height=800
)

fig.write_image('../figures/2-hour-rev.jpeg', scale=5, engine='orca')
fig